In [1]:
import os
import sys
import time

nb_dir = '/home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src'

sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

In [140]:
import pandas as pd
pd.options.mode.chained_assignment = None

from PIL import Image, ImageDraw
import copy
import warnings
warnings.filterwarnings("ignore")

In [3]:
from services.xml_document_info import (get_xml_info, get_xml_image_info)
from services import get_xml
from src.services import main
from services.preprocess import prepocess_pdf_regions
from services.get_tables import page_num_correction , get_text_table_line_df

from utilities.xml_utils import (extract_image_paths_from_pdf,
                                 extract_xml_from_digital_pdf,
                                 extract_html_bg_images_from_digital_pdf,
                                 create_directory,
                                 read_directory_files)
import config

In [ ]:
def draw_box(filepath, desired_width, desired_height, df, color="green", save=False):
    image  = Image.open(filepath)
    image  = image.resize((desired_width, desired_height))
    draw   = ImageDraw.Draw(image)
    
    for index, row in df.iterrows():
        left   = int(row['text_left'])
        right  = int(row['text_width'] + left)
        top    = int(row['text_top'])
        bottom = int(row["text_height"] + top)
        
        draw.rectangle(((left, top), (right,bottom)), outline=color)
    save_filepath = os.path.join(os.path.dirname(filepath), 'processed_' + os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

def show_df(df):
    return df.head(df.shape[0])

## start of the program

In [ ]:
'''
  folder structure of test data goes like this
  - notebooks
      - sample-data
          - input
          - output
          
  the pdfs are present in "input" directory and they are *.pdf is added into .gitignore
  just to save repo size.
'''

base_dir   = '/home/dhiraj/Documents/data'
input_dir  = os.path.join(base_dir, 'sample-data', 'input')
save_dir = os.path.join(base_dir, 'sample-data', 'bbox_output')

output_dir = os.path.join(base_dir, 'sample-data', 'output')
create_directory(input_dir)
create_directory(output_dir)

#filename   = '20695_2010_8_1501_20635_Judgement_17-Feb-2020.pdf'
filename   = 'yolo.pdf'


## extract the pdf into xml & background images for downstream processing

In [ ]:
pdf_filepath      = os.path.join(input_dir, filename)
working_dir       = os.path.join(output_dir, os.path.splitext(filename)[0])

ret               = create_directory(working_dir)

start_time        = time.time()

pdf_image_paths   = extract_image_paths_from_pdf(pdf_filepath, working_dir)
pdf_xml_dir       = extract_xml_from_digital_pdf(pdf_filepath, working_dir)
pdf_bg_image_dir  = extract_html_bg_images_from_digital_pdf(pdf_filepath, working_dir)

end_time          = time.time()
extraction_time   = end_time - start_time

In [ ]:
xml_files      = read_directory_files(pdf_xml_dir, pattern='*.xml')
bg_files       = read_directory_files(pdf_bg_image_dir, pattern='*.png')

print("Extracted xml, background images & images of file: %s" % (filename))
print('Extraction time (%f) average extraction time (%f)' % (extraction_time, extraction_time/len(bg_files)))

In [ ]:
xml_dfs, page_width, page_height = get_xml_info(xml_files[0])
img_dfs, page_width, page_height = get_xml_image_info(xml_files[0])


In [ ]:
page_index      = 4

img_filepath    = pdf_image_paths[page_index]
bg_img_filepath = bg_files[page_index]
df              = xml_dfs[page_index]
img_df          = img_dfs[page_index]


# Draw Bounding box using v_df

In [ ]:
def get_dfs(xml_dfs,img_dfs,image_files,page_height,file_name,page_index):
    multiple_pages = False
    if len(xml_dfs) > 1:
        multiple_pages =True
    header_region, footer_region = pd.DataFrame(),pd.DataFrame()#prepocess_pdf_regions(xml_dfs, page_height)
    img_df = img_dfs[page_index]
    #table_image = working_dir + '/' + page_num_correction(file_index , 3) + '.png'
    #in_df, table_df, line_df = get_text_table_line_df(table_image, xml_dfs[file_index])
    in_df = xml_dfs[page_index]
    h_df = get_xml.get_hdf(in_df, image_files, config.DOCUMENT_CONFIGS, page_index,header_region , footer_region, multiple_pages)
    v_df    = get_xml.get_vdf(h_df, config.DOCUMENT_CONFIGS)
    p_df = main.process_page_blocks(v_df, config.DOCUMENT_CONFIGS, config.BLOCK_CONFIGS)
    p_df = p_df.reset_index(drop=True)
    
    return in_df,h_df,v_df, p_df


In [ ]:
in_df,h_df,v_df, p_df = get_dfs(xml_dfs,img_dfs,bg_files,page_height,filename,page_index)

In [ ]:
#draw_box(img_filepath, page_width, page_height, p_df, color='red')

# Draw bounding box for json response 

In [126]:
from src.services.main import DocumentStructure
import os
from PIL import Image, ImageDraw

In [127]:
def draw_box(draw,page_data,attrib,color,margin,width):
    
    if page_data[attrib]:
        for block in page_data[attrib]:
                top     = block["text_top"];         left   = block["text_left"];  
                bottom  = top+ block["text_height"];  right  = left+ block["text_width"]
                draw.rectangle(((left-margin, top-margin), (right+margin,bottom+margin)), outline=color,width=width)
    return draw


def draw_bbox(data,image_files,save_path):
    for page_no in range(len(data)):
        image_path = sorted(image_files)[page_no]
        page_data = data[page_no]
        page_width = page_data['page_width']; page_height = page_data['page_height']
        image  = Image.open(image_path)
        image  = image.resize((page_width, page_height))
        draw   = ImageDraw.Draw(image)
        draw  =  draw_box(draw,page_data,attrib='text_blocks',color='red',margin=2,width=2)
        draw  =  draw_box(draw,page_data,attrib='images',color='green',margin=3,width=5)
        draw  =  draw_box(draw,page_data,attrib='tables',color='blue',margin=10,width=3)
        
        if page_data['tables'] :
            for table in page_data['tables'] :
                draw_box(draw,table,attrib='children',color=(125,0,125),margin=-10,width= 2)
#                 if table['children'] :
#                     for child in table['children'] :
#                         draw_box(draw,child,attrib='text',color=(125,0,125),margin=2,width=3)
            
        
        save_filepath = os.path.join(save_path,image_path.split('images/')[1])
        image.save(save_filepath)

In [141]:
input_dir  = '/home/dhiraj/Documents/data/sample-data/input'
output_dir = '/home/dhiraj/Documents/data/sample-data/output'
save_dir   = '/home/dhiraj/Documents/data/sample-data/bbox_output'

#filename  = '20695_2010_8_1501_20635_Judgement_17-Feb-2020.pdf'
#filename   = 'graphics_pptx.pdf'
#filename  = 'KD_CV.pdf'
#filename  = 'Madras_HC_02.pdf'
#filename  = 'ncert_physics_class11.pdf'
filename  = 'yolo.pdf'

pdf_filepath      = os.path.join(input_dir, filename)
working_dir       = os.path.join(output_dir, os.path.splitext(filename)[0])
ret               = create_directory(working_dir)
pdf_image_paths   = extract_image_paths_from_pdf(pdf_filepath, working_dir)
save_path         = save_dir+"/"+str(filename.split('.pdf')[0])
os.system('mkdir -p {0}'.format(save_path))

0

In [142]:
data = DocumentStructure(filename,input_dir)
response = data['result']

Total number of pages (10) in file (yolo.xml)
Total number of pages (10) in file (yolo.xml)


In [143]:
draw_bbox(response,pdf_image_paths,save_path)

In [16]:
a = None
            

In [17]:
len(a)

TypeError: object of type 'NoneType' has no len()

In [ ]:
response[24]

In [ ]:
ta